# Анализ базы данных крупного сервиса для чтения книг по подписке.

<div class="alert alert-info">

**Задача исследования** - проанализировать базу данных крупного сервиса для чтения книг по подписке.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные в дальнейшем помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`**
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**
Содержит данные об авторах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;
**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги. 
**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# !pip install -U psycopg2

# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Подготовка данных

In [2]:
# Формируем sql-запрос.
query1 = ''' SELECT *
            FROM books
                   '''

In [3]:
query2 = ''' SELECT *
            FROM authors
                   '''

In [4]:
query3 = ''' SELECT *
            FROM ratings
                   '''

In [5]:
query4 = ''' SELECT *
            FROM reviews
                   '''

In [6]:
# Выполняем запрос и сохраняем результат
# выполнения в DataFrame.
# Sqlalchemy автоматически установит названия колонок
# такими же, как у таблицы в БД. Нам останется только
# указать индексную колонку с помощью index_col.


books=pd.io.sql.read_sql(query1, con = engine) 
authors=pd.io.sql.read_sql(query2, con = engine)
ratings=pd.io.sql.read_sql(query3, con = engine) 
reviews=pd.io.sql.read_sql(query4, con = engine) 

In [7]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг, вышедших после 1 января 2000 года.

In [11]:
query5 = '''Select 
            COUNT (book_id)
            FROM books
            WHERE publication_date >'2000-01-01' 
          
                   '''

In [12]:
count_books_2020=pd.io.sql.read_sql(query5, con = engine)
count_books_2020

,count
0,819


<div class="alert alert-info">
    База данных содержит 819 книг, вышедших после 1 января 2000 года.

## Количество обзоров и средняя оценка для каждой книги.

In [13]:
query6 = '''Select books.title,rev.review_count, rat.rating_avg
            FROM books
            LEFT JOIN 
                (SELECT book_id, COUNT(review_id) AS review_count
                FROM reviews
                GROUP BY book_id)rev
            ON books.book_id=rev.book_id
            LEFT JOIN (SELECT book_id, AVG (rating) AS rating_avg 
                        FROM ratings 
                        GROUP BY book_id)rat
            ON books.book_id=rat.book_id
        
         
                   '''

In [14]:
review_rating=pd.io.sql.read_sql(query6, con = engine)
review_rating=review_rating.fillna(0).sort_values(by='review_count', ascending=False)
display(review_rating.head())

,title,review_count,rating_avg
161,Twilight (Twilight #1),7.0,3.662500
510,The Giver (The Giver #1),6.0,3.750000
651,Harry Potter and the Prisoner of Azkaban (Harr...,6.0,4.414634
245,The Da Vinci Code (Robert Langdon #2),6.0,3.830508
379,The Road,6.0,3.772727


<div class="alert alert-info">
Наибольшее количество обзоров на книгу Сумерки/Twilight- популярный романов американской писательницы Стефани Майер.

## Издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [15]:
query7 = '''Select publisher, COUNT(book_id) AS books_count
            FROM books INNER JOIN publishers ON books.publisher_id=publishers.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_count DESC
            LIMIT 1
                   '''

In [16]:
publisher_books=pd.io.sql.read_sql(query7, con = engine)
display(publisher_books.head())

,publisher,books_count
0,Penguin Books,42


**Наибольшее количество книг, толще 50 страниц выпустило британское издательство Penguin Books.**

## Автор с самой высокой средней оценкой книг.

In [17]:
query8 = '''Select author, AVG(rating) AS rating_avg, COUNT (ratings.book_id) AS count_rating
            FROM books INNER JOIN authors ON books.author_id= authors.author_id
                         INNER JOIN ratings ON books.book_id= ratings.book_id
            
            GROUP BY author
            HAVING COUNT(ratings.book_id)> 50
            ORDER BY rating_avg DESC
            
                   '''

In [18]:
author_rating=pd.io.sql.read_sql(query8, con = engine)
display(author_rating.head())

,author,rating_avg,count_rating
0,J.K. Rowling/Mary GrandPré,4.288462,312
1,Agatha Christie,4.283019,53
2,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
3,J.R.R. Tolkien,4.240964,166
4,Roald Dahl/Quentin Blake,4.209677,62


<div class="alert alert-info">
Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. средний рейтинг составил 4,3. Такую популярность автору принесли книги о Гарри Поттере.

## Cреднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [19]:
query9 = '''SELECT AVG(review__count) AS review__avg
            FROM (SELECT username, COUNT(review_id) AS review__count
                  FROM reviews
                  WHERE username IN (SELECT DISTINCT username
                                    FROM ratings  
                                    GROUP BY username
                                    HAVING COUNT(rating_id) >50)
                  GROUP BY username)A
         
                     
                   '''


In [20]:
review__avg=pd.io.sql.read_sql(query9, con = engine)
display(review__avg.head())

,review__avg
0,24.333333


<div class="alert alert-info">
Самые активные пользователи в среднем оставили 24 обзора на книги. 

<div class="alert alert-info">

**ВЫВОДЫ**
    
В результате анализа базы данных крупного сервиса для чтения книг по подписке были получены следующие данные: 
- База данных содержит 819 книг, вышедших после 1 января 2000 года.
- Наибольшее количество обзоров на книгу Сумерки/Twilight- популярный романов американской писательницы Стефани Майер.
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré. средний рейтинг составил 4,3. Такую популярность автору принесли книги о Гарри Поттере.
- Наибольшее количество книг, c количеством страниц более 50 выпустило британское издательство Penguin Books.
- Самые активные пользователи в среднем оставили 24 обзора на книги. 